## Ascending auction algorithm for decentralized scheduling

This is the demo notebook demonstrating the scheduling problem and the solutions given
by the ascending auction and other algorithms. It is recommended that you read the associated paper
which includes a theoretical presentation of the algorithm as well as experimental results generated with this code.

In [279]:
%load_ext autoreload
%autoreload 2
from util import generate_random_auction_agents, calculate_total_duration, calculate_std_duration
from pandas import DataFrame
from auction import *
from brute_force import *
from weighted_job_schedule import *
from priority_first import *

def run_experiment(n_exp,
                   n_agents, max_deadline, max_duration, max_utility, n_slots,
                   entry_price, min_bid,
                   agent_type = ('ascending_auction', 'brute_force', 'priority_first', 'weighted_job_first'), display_progress = False):
    results = {}
    for t in agent_type:
        results[t] = []
        results['{}_execution_time'.format(t)] = []

    results['total_duration_per_slot'] = []
    results['total_std_of_duration'] = []
    results['messages'] = []
    results['min_bid'] = []

    for i in range(n_exp):
        if display_progress:
            print("Progress: {}%".format(int(float(i)/float(n_exp)*100)))
        agents_ = generate_random_auction_agents(n_agents=n_agents,
                                                max_deadline=max_deadline,
                                                max_duration=max_duration,
                                                max_utility=max_utility)
        resource_allocation_board = DataFrame({'slot_id': arange(n_slots), 'agent': [-1]*n_slots, 'bid': [entry_price]*n_slots})

        results['total_duration_per_slot'].append(float(calculate_total_duration(agents_)) / float(n_slots))
        results['total_std_of_duration'].append(calculate_std_duration(agents_))
        results['min_bid'].append(min_bid)

        msg = 0

        if 'ascending_auction' in agent_type:
            board = AuctionBoard(resource_allocation_board.copy(), min_bid)
            start_time = time.time()
            alloc, total_utility = board.allocate_slots(agents_)
            results['ascending_auction'].append(total_utility)
            msg = board.reallocations
            results['ascending_auction_execution_time'].append(time.time() - start_time)

        if 'brute_force' in agent_type:
            board = BruteForceBoard(resource_allocation_board.copy())
            start_time = time.time()
            alloc, total_utility = board.allocate_slots(agents_)
            results['brute_force'].append(total_utility)
            results['brute_force_execution_time'].append(time.time() - start_time)

        if 'priority_first' in agent_type:
            board = PriorityBoard(resource_allocation_board.copy())
            start_time = time.time()
            alloc, total_utility = board.allocate_slots(agents_)
            results['priority_first'].append(total_utility)
            results['priority_first_execution_time'].append(time.time() - start_time)

        if 'weighted_job_first' in agent_type:
            board = WeightedJobSchedulingBoard(resource_allocation_board.copy())
            start_time = time.time()
            alloc, total_utility = board.allocate_slots(agents_)
            results['weighted_job_first'].append(total_utility)
            results['weighted_job_first_execution_time'].append(time.time() - start_time)

        results['messages'].append(msg)

    return DataFrame(results)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Generate random agents and board

The following snippet can be used to generate a random configuration of agents and task board.
The task parameters (duration, utility, deadline) are sampled from a random uniform distribution.

In [163]:
import random

from pandas import DataFrame
from util import generate_configuration

random.seed(2)

agents, df = generate_configuration(n_agents=2, max_deadline=3, max_duration=3, max_utility=10, n_slots=3, entry_price=1)
print("Agents \n")
print(DataFrame({
    "agent_id": list(map(lambda it: it.agent_id, agents)),
    "utility": list(map(lambda it: it.utility, agents)),
    "duration": list(map(lambda it: it.duration, agents)),
    "deadline": list(map(lambda it: it.deadline, agents))
}), "\n")


Agents 

   agent_id  utility  duration  deadline
0         0        4         2         2
1         1        2         1         2 



### Test ascending auction

Use the next snippet to solve the scheduling problem using ascending auction

In [167]:
from auction import AuctionBoard

board = AuctionBoard(df.copy(), 0.25)
allocations, total_utility = board.allocate_slots(agents)
print(allocations, "\n", total_utility)

   slot_id  agent   bid
0        0      0  2.25
1        1      1  2.00
2        2     -1  1.00 
 3


### Test weighted job schedule

Use the next snippet to solve the scheduling problem using weighted job schedule

In [104]:
from weighted_job_schedule import WeightedJobSchedulingBoard

board = WeightedJobSchedulingBoard(df.copy())
allocations, total_utility = board.allocate_slots(agents)
print(allocations, "\n", total_utility)

[2, 1] 
 27


### Test priority first

Use the next snippet to solve the scheduling problem using priority first

In [168]:
from priority_first import PriorityBoard

board = PriorityBoard(df.copy())
allocations, total_utility = board.allocate_slots(agents)
print(allocations, "\n", total_utility)

   slot_id  agent  bid
0        0      0    1
1        1      0    1
2        2     -1    1 
 4


### Test brute force

Use the next snippet to solve the scheduling problem using brute force (only good for small problem sizes)

In [166]:
from brute_force import BruteForceBoard

board = BruteForceBoard(df.copy())
allocations, total_utility = board.allocate_slots(agents)
print(allocations, "\n", total_utility)

   slot_id  agent  bid
0        0      0    1
1        1      0    1
2        2     -1    1 
 4


### Error Distribution

In this experiment we generate 200 configurations and apply the ascending auction algorithm,
in order to determine the deviation from the optimal solution.

In [ ]:
import pandas as pd

experiment = pd.DataFrame()
for n_agents in range(2,11):
    total = run_experiment(n_exp=200, n_agents=n_agents, max_deadline=10, max_duration=10, max_utility=30,
                       n_slots=10, entry_price=1, min_bid=0.25, display_progress=True)

    total['n_agents'] = n_agents
    total['error_auction'] = (total.brute_force - total.ascending_auction)/total.brute_force*100
    total['error_priority'] = (total.brute_force - total.priority_first)/total.brute_force*100
    total['error_weighted'] = (total.brute_force - total.weighted_job_first)/total.brute_force*100

    experiment = experiment.append(total)

print(experiment.describe())

In [268]:
import plotly.express as px
fig = px.histogram(experiment.loc[experiment['n_agents'].isin([3,5,10]), :], x='error_auction',
                   color='n_agents', labels={'error_auction':'% Error from optimal'})
fig.show()

### Mean error for different algorithms versus the number of agents
In this experiment we measure the difference of the solution (given by the algorithms) and the optimal solution.

In [293]:
m = experiment.groupby('n_agents').mean()
print(experiment.loc[experiment.n_agents == 10, :].describe())
m.reset_index(level=0, inplace=True)
m = m.melt(
    id_vars=['n_agents', 'total_duration_per_slot', 'total_std_of_duration'],
    value_vars=['error_auction', 'error_priority', 'error_weighted'],
    var_name='algorithm',
    value_name='error'
)
m = m.rename(columns={'n_agents': 'Num. agents', 'error': '% Error from optimal (mean of 200 iterations)'})
fig2 = px.line(m, x='Num. agents', y='% Error from optimal (mean of 200 iterations)', color='algorithm')
fig2.show()


       ascending_auction  ascending_auction_execution_time  brute_force  \
count         200.000000                        200.000000   200.000000   
mean           53.055000                          0.052365    56.915000   
std            18.972394                          0.033556    17.015446   
min             9.000000                          0.007002    24.000000   
25%            40.000000                          0.030006    44.000000   
50%            51.000000                          0.043017    55.000000   
75%            66.000000                          0.066246    67.250000   
max           109.000000                          0.210002   109.000000   

       brute_force_execution_time  priority_first  \
count                  200.000000      200.000000   
mean                     6.789577       46.360000   
std                      6.040511       19.585411   
min                      1.093000       19.000000   
25%                      2.939281       28.000000   
50%   

### Messages transmitted
Every time an agent bids and alters the slots, it sends a message to the board.
In this experiment we measure the transmitted messages in relation with the number of agents

In [243]:
import plotly.express as px
import pandas as pd

experiment_messages = pd.DataFrame()
for n_slots in range(10,30):
    for n_agents in range(2,30):
        print("Progress: {}/{}".format(29*(n_slots-10) + (n_agents-2), 29*20))
        total = run_experiment(n_exp=50, n_agents=n_agents, max_deadline=n_slots, max_duration=n_slots, max_utility=20,
                               n_slots=n_slots, entry_price=1, min_bid=0.25,
                               agent_type=['ascending_auction'])
        total['n_agents'] = n_agents
        total['n_slots'] = n_slots

        experiment_messages = experiment_messages.append(total)

Progress: 0/580
Progress: 1/580
Progress: 2/580
Progress: 3/580
Progress: 4/580
Progress: 5/580
Progress: 6/580
Progress: 7/580
Progress: 8/580
Progress: 9/580
Progress: 10/580
Progress: 11/580
Progress: 12/580
Progress: 13/580
Progress: 14/580
Progress: 15/580
Progress: 16/580
Progress: 17/580
Progress: 18/580
Progress: 19/580
Progress: 20/580
Progress: 21/580
Progress: 22/580
Progress: 23/580
Progress: 24/580
Progress: 25/580
Progress: 26/580
Progress: 27/580
Progress: 29/580
Progress: 30/580
Progress: 31/580
Progress: 32/580
Progress: 33/580
Progress: 34/580
Progress: 35/580
Progress: 36/580
Progress: 37/580
Progress: 38/580
Progress: 39/580
Progress: 40/580
Progress: 41/580
Progress: 42/580
Progress: 43/580
Progress: 44/580
Progress: 45/580
Progress: 46/580
Progress: 47/580
Progress: 48/580
Progress: 49/580
Progress: 50/580
Progress: 51/580
Progress: 52/580
Progress: 53/580
Progress: 54/580
Progress: 55/580
Progress: 56/580
Progress: 58/580
Progress: 59/580
Progress: 60/580
Progres

In [270]:
px.density_heatmap(experiment_messages,
                   x='n_agents', y='n_slots', z='messages',
                   labels={'n_agents': 'Num. agents', 'n_slots': 'Num. slots'})

### Running time
We measure the execution time of ascending auction with different number of agents and slots

In [278]:
fig2 = px.scatter(experiment_messages.loc[experiment_messages.n_slots == 10, :].groupby('n_agents').mean().reset_index(level=0),
                  x='n_agents', y='ascending_auction_execution_time',
                  labels={'n_agents': 'Num. Agents', 'ascending_auction_execution_time': 'Execution time (mean of 50 iterations)'})
fig2.show()

experiment_running_time_multiple_agents = pd.DataFrame()
for n_agents in [5, 10, 20, 30]:
    total = run_experiment(n_exp=50, n_agents=n_agents, max_deadline=10, max_duration=10, max_utility=20,
                           n_slots=10, entry_price=1, min_bid=0.25,
                           agent_type=['ascending_auction', 'priority_first'])
    total['n_agents'] = n_agents
    total['n_slots'] = n_slots

    experiment_running_time_multiple_agents = experiment_running_time_multiple_agents.append(total)

experiment_running_time_multiple_agents = experiment_running_time_multiple_agents.groupby('n_agents').mean()
experiment_running_time_multiple_agents.reset_index(level=0, inplace=True)
experiment_running_time_multiple_agents = experiment_running_time_multiple_agents.melt(
    id_vars=['n_agents', 'total_duration_per_slot', 'total_std_of_duration'],
    value_vars=['ascending_auction_execution_time', 'priority_first_execution_time'],
    var_name='algorithm',
    value_name='time'
)

fig3 = px.bar(experiment_running_time_multiple_agents, x='n_agents', y='time', barmode='group', color='algorithm',
              labels={'n_agents': 'Num. agents'})
fig3.show()

### Running time and total utility vs min_bid

In [281]:
experiment_min_bid = pd.DataFrame()
for min_bid in [0.05, 0.1, 0.2, 0.4, 0.6, 1, 1.5, 2, 2.5, 3]:
    total = run_experiment(n_exp=50, n_agents=10, max_deadline=10, max_duration=10, max_utility=20,
                           n_slots=10, entry_price=1, min_bid=min_bid,
                           agent_type=['ascending_auction'])
    total['n_agents'] = n_agents
    total['n_slots'] = n_slots

    experiment_min_bid = experiment_min_bid.append(total)

In [289]:
fig4 = px.scatter(experiment_min_bid,
                  x='ascending_auction_execution_time', y='ascending_auction',
                  color='min_bid', log_x=True, log_y=True,
                  labels={'ascending_auction_execution_time': 'Running time (log scale)', 'ascending_auction': 'Total utility (log scale)'})
fig4.show()

fig5 = px.scatter(experiment_min_bid.groupby('min_bid').mean().reset_index(level=0), x='ascending_auction_execution_time', y='ascending_auction',
                  color='min_bid',
                  labels={'ascending_auction_execution_time': 'Running time (mean of 50 iterations)', 'ascending_auction': 'Total utility (mean of 50 iterations)'})

fig5.data[0].update(mode='markers+lines')
fig5.show()
